In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import OrderedDict
import time
from sklearn.metrics import mean_squared_error,roc_auc_score,mean_absolute_error,log_loss
import sys
sys.path.append('../benchmark/')
from lvxnn_test import lvxnn
from xgb_test import xgb
from svd_test import svd
from deepfm_fm_test import deepfm_fm
import sys
sys.path.append('../../')
from lvxnn.LVXNN import LV_XNN
from lvxnn.DataReader import data_initialize

import tensorflow as tf
tf.compat.v1.enable_eager_execution()
random_state=0
data= pd.read_csv('../santander/data/train_new.csv')
train , test = train_test_split(data,test_size=0.2,random_state=1)

task_type = "Classification"

list1 = train.columns
meta_info = OrderedDict()
for i in list1:
    meta_info[i]={'type': 'categorical','source':'user'}
meta_info['income']={"type":"continues",'source':'user'}
meta_info['cust_seniority']={"type":"continues",'source':'user'}
meta_info['age'] = {"type":"continues",'source':'user'}
meta_info['item'] = {'type': 'categorical','source':'item'}
meta_info['user_id']={"type":"id",'source':'user'}
meta_info['item_id']={"type":"id",'source':'item'}
meta_info['target']={"type":"target",'source':''}

lx_params = {
        "main_effect_epochs":20,
        "interaction_epochs" : 20 ,
        "tuning_epochs" : 10 , 
        "mf_training_iters": 15,
        "u_group_num": 50,
        "i_group_num":10
    }

tr_x, tr_Xi, tr_y, tr_idx, te_x, te_Xi, te_y, val_x, val_Xi, val_y, val_idx, meta_info, model_info = data_initialize(train,test,meta_info,task_type ,'warm', random_state)

W0701 10:46:47.016756 11600 deprecation.py:323] From C:\Users\64161\Anaconda3\lib\site-packages\tensorflow_core\python\compat\v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


Memory usage of dataframe is 108.45 MB


../..\lvxnn\DataReader.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype('category')
../..\lvxnn\DataReader.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype(np.int8)
../..\lvxnn\DataReader.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].as

Memory usage after optimization is: 23.42 MB
Decreased by 78.4%
Memory usage of dataframe is 27.11 MB
Memory usage after optimization is: 5.86 MB
Decreased by 78.4%
test cold start user: 0
test cold start item: 0
validation cold start user: 0
validation cold start item: 0


In [8]:
[(te_Xi[:,1] != 'cold') & (te_Xi[:,0] != 'cold')] == [True]

C:\Users\64161\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


True

In [ ]:
#result_lvxnn = lvxnn('warm',tr_x, val_x, te_x, tr_y, val_y, te_y,tr_Xi, val_Xi, te_Xi, tr_idx, val_idx, meta_info, model_info, task_type , random_state=0, params=lx_params)
result_svd = svd('warm',tr_x, val_x, te_x, tr_y, val_y, te_y,tr_Xi, val_Xi, te_Xi, tr_idx, val_idx, meta_info, model_info, task_type , random_state=0)
result_deepfm, result_fm = deepfm_fm('warm',train,test,tr_x, val_x, te_x, tr_y, val_y, te_y,tr_Xi, val_Xi, te_Xi, tr_idx, val_idx, meta_info, model_info, task_type , random_state=0, epochs=10)
result_xgb = xgb('warm',tr_x, val_x, te_x, tr_y, val_y, te_y,tr_Xi, val_Xi, te_Xi, tr_idx, val_idx, meta_info, model_info, task_type , random_state=0)


result_sim_std = pd.concat([result_lvxnn,result_svd,result_xgb,result_deepfm,result_fm],0)

result_sim_std.to_csv('santander_result.csv',index=None)

../benchmark\svd_test.py:101: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if [(te_Xi[:,1] != 'cold') & (te_Xi[:,0] != 'cold')] != [True]:
../benchmark\svd_test.py:101: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if [(te_Xi[:,1] != 'cold') & (te_Xi[:,0] != 'cold')] != [True]:
../benchmark\svd_test.py:101: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if [(te_Xi[:,1] != 'cold') & (te_Xi[:,0] != 'cold')] != [True]:
../benchmark\svd_test.py:101: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if [(te_Xi[:,1] != 'cold') & (te_Xi[:,0] != 'cold')] != [True]:
../benchmark\svd_test.py:101: FutureWarning: elementwise comparison failed; returning scalar instead, but in the

{'embedding_size': 5, 'deep_layers': [32, 32], 'use_deep': True, 'use_fm': True, 'deep_layers_activation': <function relu at 0x0000021207F1A158>, 'loss_type': 'mse', 'epoch': 10, 'batch_size': 1024, 'learning_rate': 0.01, 'optimizer_type': 'adam', 'batch_norm': 0, 'batch_norm_decay': 0.995, 'l2_reg': 0.01, 'verbose': False, 'eval_metric': <function mean_absolute_error at 0x0000021206748950>, 'random_seed': 0, 'feature_size': 36941, 'field_size': 20}


W0701 10:48:32.102771 11600 deprecation.py:506] From ../benchmark/deepfm\DeepFM.py:93: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
../benchmark\deepfm_fm_test.py:176: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if [(te_Xi[:,1] != 'cold') & (te_Xi[:,0] != 'cold')] != [True]:


{'embedding_size': 5, 'deep_layers': [32, 32], 'use_deep': True, 'use_fm': True, 'deep_layers_activation': <function relu at 0x0000021207F1A158>, 'loss_type': 'mse', 'epoch': 10, 'batch_size': 1024, 'learning_rate': 0.01, 'optimizer_type': 'adam', 'batch_norm': 0, 'batch_norm_decay': 0.995, 'l2_reg': 0.01, 'verbose': False, 'eval_metric': <function mean_absolute_error at 0x0000021206748950>, 'random_seed': 0, 'feature_size': 36941, 'field_size': 20}


../benchmark\deepfm_fm_test.py:176: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if [(te_Xi[:,1] != 'cold') & (te_Xi[:,0] != 'cold')] != [True]:


...

{'embedding_size': 5, 'deep_layers': [32, 32], 'use_deep': True, 'use_fm': True, 'deep_layers_activation': <function relu at 0x0000021207F1A158>, 'loss_type': 'mse', 'epoch': 10, 'batch_size': 1024, 'learning_rate': 0.01, 'optimizer_type': 'adam', 'batch_norm': 0, 'batch_norm_decay': 0.995, 'l2_reg': 0.01, 'verbose': False, 'eval_metric': <function mean_absolute_error at 0x0000021206748950>, 'random_seed': 0, 'feature_size': 36941, 'field_size': 20}


../benchmark\deepfm_fm_test.py:176: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if [(te_Xi[:,1] != 'cold') & (te_Xi[:,0] != 'cold')] != [True]:


{'embedding_size': 5, 'deep_layers': [32, 32], 'use_deep': True, 'use_fm': True, 'deep_layers_activation': <function relu at 0x0000021207F1A158>, 'loss_type': 'mse', 'epoch': 10, 'batch_size': 1024, 'learning_rate': 0.01, 'optimizer_type': 'adam', 'batch_norm': 0, 'batch_norm_decay': 0.995, 'l2_reg': 0.01, 'verbose': False, 'eval_metric': <function mean_absolute_error at 0x0000021206748950>, 'random_seed': 0, 'feature_size': 36941, 'field_size': 20}


../benchmark\deepfm_fm_test.py:176: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if [(te_Xi[:,1] != 'cold') & (te_Xi[:,0] != 'cold')] != [True]:


{'embedding_size': 5, 'deep_layers': [32, 32], 'use_deep': True, 'use_fm': True, 'deep_layers_activation': <function relu at 0x0000021207F1A158>, 'loss_type': 'mse', 'epoch': 10, 'batch_size': 1024, 'learning_rate': 0.01, 'optimizer_type': 'adam', 'batch_norm': 0, 'batch_norm_decay': 0.995, 'l2_reg': 0.01, 'verbose': False, 'eval_metric': <function mean_absolute_error at 0x0000021206748950>, 'random_seed': 0, 'feature_size': 36941, 'field_size': 20}
